Adınız:  Umutcan

Soyadınız:  Kemahlı

Okul Numaranız:  2212721050

GitHub Repo Bağlantısı: https://github.com/umutckem/MLP_Flask.git

Bu kod bloğunda, Google Colab ortamı Google Drive ile ilişkilendirilmiş ve proje kapsamında kullanılacak olan Medical Insurance Cost veri seti Colab ortamına aktarılmıştır.
Ardından pandas kütüphanesi kullanılarak CSV formatındaki veri seti bir DataFrame yapısına dönüştürülmüş, veri setine ait kolon isimleri listelenmiş ve ilk beş satır görüntülenerek veri yapısı hakkında genel bir ön inceleme yapılmıştır.

In [ ]:
# Google Drive'ı Colab ortamına bağlamak için gerekli modülü içe aktarıyoruz
from google.colab import drive

# Drive'ı /content/drive dizinine mount ederek erişilebilir hale getiriyoruz
drive.mount('/content/drive')

# Veri işleme için gerekli kütüphaneleri yüklüyoruz: pandas ve numpy
import pandas as pd
import numpy as np

# Kullanacağımız veri setinin Google Drive içindeki dosya yolu
# (Bu yol kullanıcı tarafından belirlenmiştir)
csv_path = "/content/drive/MyDrive/Dersler/Makine-Ogrenmesi/Odev/Uygulama-3/dataset/insurance.csv"

# CSV dosyasını pandas ile DataFrame olarak okuyoruz
df = pd.read_csv(csv_path)

# Veri setindeki kolon isimlerini ekrana yazdırıyoruz
print("Kolonlar:", df.columns.tolist())

# Veri setinin ilk 5 satırını göstererek veri yapısını incelemeyi sağlıyoruz
display(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kolonlar: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


df.info() fonksiyonu, veri setinin yapısını ve eksik değer durumunu analiz etmek için kullanılmıştır.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


df.head() fonksiyonu ile veri setinin ilk satırları incelenerek veri yapısı hakkında ön gözlem yapılmıştır.

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


df.isnull().sum() kullanılarak veri setindeki eksik değerler sütun bazında analiz edilmiştir.

In [ ]:
df.isnull().sum()

,0
age,5
sex,2
bmi,5
children,2
smoker,3
region,3
charges,0


Bu kod bloğunda, veri setinde bulunan eksik değerleri doldurmak amacıyla sayısal ve kategorik değişkenler için farklı yaklaşımlar uygulanmıştır.

Öncelikle, orijinal veri setinin korunması amacıyla df.copy() fonksiyonu kullanılarak veri setinin bir kopyası oluşturulmuştur.
Sayısal değişkenlerde (numeric_cols) bulunan eksik değerler, uç değerlerden daha az etkilendiği için medyan (median) yöntemi ile doldurulmuştur.

Kategorik değişkenlerde (categorical_cols) ise eksik değerler, veri setinde en sık tekrar eden değer olan mod (mode) kullanılarak doldurulmuştur.

Bu yaklaşım sayesinde veri setinin genel dağılımı korunmuş ve model eğitimi için eksiksiz bir veri seti elde edilmiştir.

In [ ]:
df_filled = df.copy()

# Sayısal sütunlar: median
for col in numeric_cols:
    median_value = df_filled[col].median()
    df_filled[col] = df_filled[col].fillna(median_value)

# Kategorik sütunlar: mode
for col in categorical_cols:
    mode_value = df_filled[col].mode()[0]
    df_filled[col] = df_filled[col].fillna(mode_value)


df.isnull().sum() kullanılarak veri setindeki eksik değerler sütun bazında analiz edilmiştir.

In [ ]:
df_filled.isnull().sum()

,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0


In [ ]:
filled_path = "/content/drive/MyDrive/Dersler/Makine-Ogrenmesi/Odev/Uygulama-3/dataset/insurance_filled.csv"
df_filled.to_csv(filled_path, index=False)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import statsmodels.api as sm

In [ ]:
csv_path = "/content/drive/MyDrive/Dersler/Makine-Ogrenmesi/Odev/Uygulama-3/dataset/insurance_filled.csv"
df = pd.read_csv(csv_path)

Bu adımda, modelin eğitimi için kullanılacak olan ilgili öznitelikler (feature’lar) belirlenmiştir.
selected_features listesi içerisinde, sigorta maliyetini etkilediği düşünülen değişkenler ile hedef değişken (charges) tanımlanmıştır.

df = df[selected_features] ifadesi kullanılarak, veri seti yalnızca seçilen bu sütunları içerecek şekilde daraltılmıştır.
Bu işlem sayesinde gereksiz veya modele katkı sağlamayan değişkenler veri setinden çıkarılmış, daha sade ve kontrollü bir veri yapısı elde edilmiştir.

In [ ]:
selected_features = [
    "age",
    "bmi",
    "children",
    "sex",
    "smoker",
    "region",
    "charges"   # hedef değişken
]

df = df[selected_features]

u adımda, veri setinde yer alan değişkenler sayısal (numeric) ve kategorik (categorical) olmak üzere iki gruba ayrılmıştır.

numeric_cols listesi, matematiksel işlemlere doğrudan uygun olan ve sayısal değerler içeren değişkenleri (age, bmi, children) tanımlamaktadır.
categorical_cols listesi ise sınıflandırma veya etiket bilgisi içeren, sayısal olmayan değişkenleri (sex, smoker, region) belirtmektedir.

Bu ayrım, eksik değer doldurma, kodlama (encoding) ve modelleme aşamalarında her değişken türüne uygun yöntemlerin uygulanabilmesi için yapılmıştır.

In [ ]:
categorical_cols = ["sex", "smoker", "region"]
numeric_cols = ["age", "bmi", "children"]

Bu adımda, sayısal olmayan kategorik değişkenler (sex, smoker, region), makine öğrenmesi modelinde kullanılabilmeleri için One-Hot Encoding yöntemi ile sayısal forma dönüştürülmüştür.

pd.get_dummies() fonksiyonu kullanılarak her bir kategorik değişken için ayrı ikili (0–1) sütunlar oluşturulmuştur.
drop_first=True parametresi ise dummy variable trap (çoklu doğrusal bağlantı) probleminden kaçınmak amacıyla her kategorik değişken için bir referans sınıfın dışarıda bırakılmasını sağlamaktadır.

Bu işlem sonucunda, modelin kategorik verileri doğru ve anlamlı bir şekilde öğrenebilmesi sağlanmıştır.

In [ ]:
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

Bu adımda, modelin eğitimi için bağımsız değişkenler (X) ve bağımlı değişken (y) birbirinden ayrılmıştır.

X değişkeni, hedef değişken olan charges sütunu çıkarılarak oluşturulmuş ve modelin tahmin yaparken kullanacağı tüm giriş özelliklerini içermektedir.
y değişkeni ise tahmin edilmek istenen hedef değişken olan charges sütununu temsil etmektedir.

Ayrıca, charges sütunu astype(float) ile sayısal veri tipine dönüştürülerek regresyon modelinin gerektirdiği uygun format sağlanmıştır.

In [ ]:
X = df_encoded.drop("charges", axis=1)
y = df_encoded["charges"].astype(float)

Bu adımda, One-Hot Encoding işlemi sonrasında elde edilen tüm sütunlar astype(float) kullanılarak sayısal (float) veri tipine dönüştürülmüştür.

Bu dönüşüm, özellikle statsmodels kütüphanesi ile kurulan OLS regresyon modelinin yalnızca sayısal veri tipleri ile çalışması gerekliliğinden dolayı yapılmıştır.
Böylece veri tipi uyumsuzluklarından kaynaklanabilecek hatalar önlenmiş ve Backward Elimination sürecinin sorunsuz şekilde yürütülmesi sağlanmıştır.

In [ ]:
# TÜM DATAFRAME'İ FLOAT'A ZORLA
df_encoded = df_encoded.astype(float)

Bu adımda, makine öğrenmesi modelinin eğitimi için bağımsız değişkenler (X) ve bağımlı değişken (y) tanımlanmıştır.

X değişkeni, hedef değişken olan charges sütunu çıkarılarak oluşturulmuş ve modelin tahmin yaparken kullanacağı tüm giriş özelliklerini içermektedir.
y değişkeni ise tahmin edilmek istenen çıktı değişkeni olan charges sütununu temsil etmektedir.

Bu ayrım, regresyon modelinin doğru bir şekilde eğitilebilmesi için temel bir adımdır.

In [ ]:
X = df_encoded.drop("charges", axis=1)
y = df_encoded["charges"]

Bu kod bloğunda, çoklu doğrusal regresyon modelinde istatistiksel olarak anlamsız öznitelikleri belirlemek ve modelden çıkarmak amacıyla Backward Elimination yöntemi uygulanmıştır.

İlk olarak sm.add_constant(X) fonksiyonu ile modele sabit terim (intercept) eklenmiştir. Ardından, tüm değişkenleri içeren başlangıç modeli OLS yöntemi ile kurulmuştur.

Model kurulduktan sonra her bir değişkene ait p-value değerleri incelenmiştir. Anlamlılık seviyesi olarak 0.05 kabul edilmiştir.
p-value değeri 0.05’ten büyük olan değişkenler istatistiksel olarak anlamsız kabul edilerek, en yüksek p-value’ya sahip değişken modelden çıkarılmıştır.

Bu işlem, modelde kalan tüm değişkenlerin p-value değerleri 0.05’ten küçük veya eşit olana kadar döngü içerisinde tekrarlanmıştır.
Sonuç olarak, yalnızca istatistiksel olarak anlamlı özniteliklerden oluşan daha sade ve güçlü bir regresyon modeli elde edilmiştir.

In [ ]:
X_sm = sm.add_constant(X)
cols = list(X_sm.columns)

while True:
    model = sm.OLS(y, X_sm).fit()
    p_values = model.pvalues

    max_p = p_values.max()

    if max_p > 0.05:
        remove_col = p_values.idxmax()
        cols.remove(remove_col)
        X_sm = X_sm[cols]
    else:
        break


Backward Elimination yöntemi tamamlandıktan sonra, modelde kalan istatistiksel olarak anlamlı öznitelikler kullanılarak nihai çoklu doğrusal regresyon modeli oluşturulmuştur.

sm.OLS(y, X_sm).fit() ifadesi ile son model eğitilmiş,
final_model.summary() fonksiyonu ile modelin detaylı istatistiksel özeti elde edilmiştir.

Bu özet tabloda;

Her değişkene ait katsayılar (coefficients),

p-value değerleri,

Modelin genel başarısını gösteren R² ve Adjusted R² değerleri,

Modelin istatistiksel anlamlılığını gösteren F-statistic

gibi önemli bilgiler yer almaktadır.

Bu adım, oluşturulan modelin hem bireysel değişkenler hem de genel performans açısından değerlendirilmesini sağlamaktadır.

In [ ]:
final_model = sm.OLS(y, X_sm).fit()
final_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     968.4
Date:                Tue, 16 Dec 2025   Prob (F-statistic):               0.00
Time:                        06:03:29   Log-Likelihood:                -13566.
No. Observations:                1338   AIC:                         2.714e+04
Df Residuals:                    1333   BIC:                         2.717e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.221e+04    957.327    -12.758      0.000   -1.41e+04   -1.03e+04
age          258.1851     12.052     21.423      0.000     234.543     281.827
bmi          325.6347     27.767     11.727      0.000     271.162     380.107
children     485.2171    139.406      3.481      0.001     211.739     758.696
smoker_yes  2.376e+04    416.453     57.061      0.000    2.29e+04    2.46e+04
==============================================================================
Omnibus:                      321.880   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              821.038
Skew:                           1.268   Prob(JB):                    5.17e-179
Kurtosis:                       5.880   Cond. No.                         293.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Bu adımda, veri seti modelin performansını değerlendirebilmek amacıyla eğitim (train) ve test olmak üzere iki parçaya ayrılmıştır.

train_test_split() fonksiyonu kullanılarak:

Verinin %80’i eğitim,

%20’si test verisi olarak ayrılmıştır (test_size=0.2).

Backward Elimination sırasında eklenen sabit terim (const) modeli eğitirken gerekli olmadığı için X_sm.drop("const", axis=1) ile giriş değişkenlerinden çıkarılmıştır.

random_state=42 parametresi ise işlemin tekrarlanabilir olmasını sağlamak için kullanılmıştır.

Bu işlem sayesinde model, daha önce görmediği test verileri üzerinde değerlendirilerek genelleme başarısı ölçülebilir hale gelmiştir.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_sm.drop("const", axis=1),
    y,
    test_size=0.2,
    random_state=42
)


Bu adımda, LinearRegression sınıfı kullanılarak çoklu doğrusal regresyon modeli oluşturulmuş ve eğitim verileri ile model eğitilmiştir.

lr = LinearRegression() ifadesi ile regresyon modeli tanımlanmış,
lr.fit(X_train, y_train) fonksiyonu ile model, eğitim veri setindeki bağımsız değişkenler ile hedef değişken arasındaki ilişkiyi öğrenmiştir.

Bu işlem sonucunda model, giriş değişkenlerine ait katsayıları hesaplayarak tahmin yapabilir hale gelmiştir.

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

Bu adımda, eğitilen çoklu doğrusal regresyon modeli kullanılarak test veri seti üzerinde tahminler yapılmıştır.
lr.predict(X_test) fonksiyonu ile modelin ürettiği tahmin değerleri elde edilmiştir.

Model performansını değerlendirmek amacıyla üç farklı metrik hesaplanmıştır:

R² (R Kare): Bağımsız değişkenlerin, hedef değişkendeki varyansın ne kadarını açıkladığını gösterir. Değerin 1’e yakın olması modelin başarısını ifade eder.

MAE (Mean Absolute Error): Gerçek değerler ile tahmin edilen değerler arasındaki mutlak hata ortalamasını verir.

MSE (Mean Squared Error): Hataların karelerinin ortalamasını alarak büyük hatalara daha fazla ağırlık verir.

Bu metrikler sayesinde modelin doğruluğu ve hata düzeyi nicel olarak değerlendirilmiştir.

In [ ]:
y_pred = lr.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R² Score:", r2)
print("MAE:", mae)
print("MSE:", mse)


R² Score: 0.754242208226595
MAE: 4314.872874028146
MSE: 38153585.427040525


In [ ]:
import pickle

Bu adımda, eğitilen çoklu doğrusal regresyon modeli daha sonra kullanılmak üzere .pkl formatında kaydedilmiştir.
pickle.dump() fonksiyonu kullanılarak model nesnesi belirtilen dosya yoluna yazılmıştır.

Bu işlem sayesinde model:

Yeniden eğitilmeye gerek kalmadan,

Flask tabanlı web uygulamasında veya farklı ortamlarda
kolayca tekrar kullanılabilir hale getirilmiştir.

In [ ]:
model_path = "/content/drive/MyDrive/Dersler/Makine-Ogrenmesi/Odev/Uygulama-3/model.pkl"

with open(model_path, "wb") as f:
    pickle.dump(lr, f)

print("✔ model.pkl kaydedildi:", model_path)


✔ model.pkl kaydedildi: /content/drive/MyDrive/Dersler/Makine-Ogrenmesi/Odev/Uygulama-3/model.pkl


Bu adımda, eğitilen regresyon modelinin beklediği giriş değişkenlerinin (feature) isimleri ve sırası kaydedilmiştir.
X_train.columns.tolist() ifadesi ile model eğitiminde kullanılan sütun isimleri bir liste haline getirilmiş ve pickle.dump() fonksiyonu ile .pkl formatında saklanmıştır.

Bu işlem, özellikle Flask uygulamasında kullanıcıdan alınan verilerin modele doğru sırada ve doğru kolon isimleriyle verilmesini sağlamak için yapılmıştır.
Böylece model ile arayüz arasında tutarlılık korunmuş ve hatalı tahminlerin önüne geçilmiştir.

In [ ]:
columns_path = "/content/drive/MyDrive/Dersler/Makine-Ogrenmesi/Odev/Uygulama-3/model_columns.pkl"

with open(columns_path, "wb") as f:
    pickle.dump(X_train.columns.tolist(), f)

print("✔ model_columns.pkl kaydedildi:", columns_path)


✔ model_columns.pkl kaydedildi: /content/drive/MyDrive/Dersler/Makine-Ogrenmesi/Odev/Uygulama-3/model_columns.pkl
